In [283]:
%pip install yfiles_jupyter_graphs


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [284]:
from yfiles_jupyter_graphs import GraphWidget
import xml.etree.ElementTree as ET
import os 

In [285]:
namespaces = {
  'ODM': "http://www.cdisc.org/ns/odm/v1.3",
  'SDM':  "http://www.cdisc.org/ns/studydesign/v1.0"
}

def fully_qualified(ns, element):
  return "{%s}%s" % (ns, element)

def remove_ns(name):
  for k, v in namespaces.items():
    if v in name:
      return name.replace("{%s}" % (v), "")
  return name

node_types = { 
  fully_qualified(namespaces['ODM'], "ODM"): "root"
}
ignore_nodes = [ 
  fully_qualified(namespaces['SDM'], "Summary"), 
  fully_qualified(namespaces['ODM'], "Description"), 
  fully_qualified(namespaces['ODM'], "ItemGroupDef"),
  fully_qualified(namespaces['ODM'], "ItemGroupRef"),
  fully_qualified(namespaces['ODM'], "ItemDef") 
]
edge_nodes = [ 
  fully_qualified(namespaces['SDM'], "ActivityRef"), 
  fully_qualified(namespaces['ODM'], "FormRef"),
  fully_qualified(namespaces['SDM'], "ArmRef") 
]
preserve_oid_nodes = [ 
  fully_qualified(namespaces['SDM'], "ActivityDef"), 
  fully_qualified(namespaces['ODM'], "FormDef"),
  fully_qualified(namespaces['SDM'], "Arm") 
]
link_by_oid_nodes = {
  fully_qualified(namespaces['SDM'], "Transition"): { 'attribute': 'SourceActivityOID', 'label': 'source' }
}
  

In [286]:
nodes = []
node_index = 0
edges = []
edge_index = 0
node_oid_map = {}
save_edges = []

def add_node(label, properties, node_type=None):
  global node_index
  global nodes
  node_index += 1
  properties['label'] = label
  if not node_type == None:
    properties['node_type'] = node_type
  nodes.append({ 'id': node_index, 'properties': properties })
  return node_index

def add_edge(label, start, end):
  global edge_index
  global edges
  edge_index += 1
  edges.append( {id: edge_index, 'start': start, 'end': end, 'properties': { 'label': label } })
  return edge_index

def save_edge(index, oid, label):
  global save_edges
  if not index == None:
    save_edges.append({ 'start': index, 'end_oid': oid, 'label': label })

def save_oid_def(oid, node_index):
  global node_oid_map
  node_oid_map[oid] = node_index

def find_oid_ref(properties):
  for k,v in properties.items():
    if "OID" in k:
      return v
  return None


In [287]:

def process_element(element, parent_index=None):
  element_index = None
  node_type = None
  if element.tag in edge_nodes:
    attribs = element.attrib
    label = remove_ns(element.tag)
    oid = find_oid_ref(attribs)
    save_edge(parent_index, oid, label)
  else:
    attribs = element.attrib
    label = remove_ns(element.tag)
    if element.tag in node_types:
      node_type = node_types[element.tag]
    element_index = add_node(label, attribs, node_type)
  if element.tag in preserve_oid_nodes:
    save_oid_def(attribs['OID'], element_index)
  if element.tag in link_by_oid_nodes:
    info = link_by_oid_nodes[element.tag]
    save_edge(element_index, attribs[info['attribute']], info['label'])
  for child in element:
    if not child.tag in ignore_nodes:
      child_index = process_element(child, element_index)
      if not child_index == None:
        child_edge_index = add_edge("", element_index, child_index)
  return element_index

In [288]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/lzzt_trial.xml")
tree = ET.parse(file_path)

root = tree.getroot()

for item in root.findall('.'):
  process_element(item)

for edge in save_edges:
  oid = edge['end_oid']
  if oid in node_oid_map:
    add_edge(edge['label'], edge['start'], node_oid_map[oid])
  else:
    print("Error: Cannot find oid %s for label %s" % (oid, edge['label']))
    
print("NODES:", nodes)
print("")
print("")
print("EDGES:", edges)
print("")
print("")
print("OID_MAP:", node_oid_map)
print("")
print("")
print("SAVE_EDGES:", save_edges)


NODES: [{'id': 1, 'properties': {'CreationDateTime': '2010-12-08T14:31:28+01:00', 'Description': 'LZZT Study Design, CDISC ODM version 1.3 format', 'FileOID': 'LZZT_STUDY', 'FileType': 'Snapshot', 'Granularity': 'Metadata', 'ODMVersion': '1.3', 'SourceSystem': 'XML4Pharma CDISC ODM Study Designer', 'SourceSystemVersion': '2010-R2', 'label': 'ODM', 'node_type': 'root'}}, {'id': 2, 'properties': {'OID': 'LZZT', 'label': 'Study'}}, {'id': 3, 'properties': {'label': 'GlobalVariables'}}, {'id': 4, 'properties': {'label': 'StudyName'}}, {'id': 5, 'properties': {'label': 'StudyDescription'}}, {'id': 6, 'properties': {'label': 'ProtocolName'}}, {'id': 7, 'properties': {'label': 'BasicDefinitions'}}, {'id': 8, 'properties': {'Description': 'LZZT study design version 1', 'Name': 'LZZT study design version 1', 'OID': 'LZZT_1', 'label': 'MetaDataVersion'}}, {'id': 9, 'properties': {'label': 'Protocol'}}, {'id': 10, 'properties': {'Mandatory': 'Yes', 'StudyEventOID': 'SE.SCREENING_VISIT', 'label': 

In [289]:
def custom_node_color(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'root':
      return 'black'
    elif node['properties']['node_type'] == 'anchor':
      return '#999999'
    elif node['properties']['node_type'] == 'condition':
      return '#999999'
    elif node['properties']['node_type'] == 'timepoint':
      return '#555555'
    elif node['properties']['node_type'] == 'visit':
      return '#c1141a'
    elif node['properties']['node_type'] == 'activity':
      return '#1555bd'
    elif node['properties']['node_type'] == 'bc':
      return '#c0d6e4'
    else:
      return 'white'
  else: 
    return 'white'

def custom_node_style(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'root':
      return { 'shape': 'round-rectangle' }
    elif node['properties']['node_type'] == 'anchor':
      return { 'shape': 'hexagon2' }
    elif node['properties']['node_type'] == 'condition':
      return { 'shape': 'diamond' }
    elif node['properties']['node_type'] == 'timepoint':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'visit':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'activity':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'bc':
      return { 'shape': 'circle' }
    else:
      return { 'shape': 'circle' }
  else: 
    return { 'shape': 'circle' }


In [290]:
w = GraphWidget()
w.set_directed(True)
w.hierarchic_layout()

w.set_nodes(nodes)
w.set_edges(edges)

w.set_node_color_mapping(custom_node_color)
w.set_node_styles_mapping(custom_node_style)
w

GraphWidget(layout=Layout(height='500px', width='100%'))